In [2]:
!pip install opencv-python
!pip install keras
!pip install tensorflow

  Using cached opencv_python-4.8.1.78-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
Using cached opencv_python-4.8.1.78-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (61.7 MB)
  Using cached keras-3.0.1-py3-none-any.whl.metadata (4.8 kB)
  Using cached absl_py-2.0.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached rich-13.7.0-py3-none-any.whl.metadata (18 kB)
  Using cached namex-0.0.7-py3-none-any.whl (5.8 kB)
  Using cached dm_tree-0.1.8-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (152 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)
Using cached keras-3.0.1-py3-none-any.whl (999 kB)
Using cached absl_py-2.0.0-py3-none-any.whl (130 kB)
Using cached rich-13.7.0-py3-none-any.whl (240 kB)
Using cached markdown_it_py-3.0.0-py3-none-any.whl (87 kB)
  Using cached tensorflow-2.15.0.post1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4

In [3]:
import numpy as np
import cv2

import matplotlib.pyplot as plt
from keras.datasets import cifar10

from tmu.models.classification.vanilla_classifier import TMClassifier
from time import time

2023-12-10 21:21:35.486081: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-10 21:21:35.486129: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-10 21:21:35.487909: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-10 21:21:35.496684: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-10 21:21:36.375617: W tensorflow/comp

In [7]:
device = "GPU"
max_included_literals = 32
resolution = 8
factor = 1

In [8]:
import cv2
from matplotlib import pyplot as plt
from keras.datasets import cifar10

# Load CIFAR-10 dataset
(X_train_org, Y_train), (X_test_org, Y_test) = cifar10.load_data()

Y_train = Y_train.reshape(Y_train.shape[0])
Y_test = Y_test.reshape(Y_test.shape[0])


X_train_thermometer = np.empty((X_train_org.shape[0], X_train_org.shape[1], X_train_org.shape[2], X_train_org.shape[3], resolution), dtype=np.uint8)
for z in range(resolution):
    X_train_thermometer[:, :, :, :, z] = X_train_org[:, :, :, :] >= (z + 1) * 255 / (resolution + 1)

X_test_thermometer = np.empty((X_test_org.shape[0], X_test_org.shape[1], X_test_org.shape[2], X_test_org.shape[3], resolution), dtype=np.uint8)
for z in range(resolution):
    X_test_thermometer[:, :, :, :, z] = X_test_org[:, :, :, :] >= (z + 1) * 255 / (resolution + 1)

X_train_thermometer = X_train_thermometer.reshape((X_train_org.shape[0], X_train_org.shape[1], X_train_org.shape[2], 3 * resolution))
X_test_thermometer = X_test_thermometer.reshape((X_test_org.shape[0], X_test_org.shape[1], X_test_org.shape[2], 3 * resolution))

In [ ]:
from tmu.models.classification.vanilla_classifier import TMClassifier
from time import time

num_clauses=2000 * factor
T=1500 * factor
s=2.5
patch_size=2
max_included_literals=32
weighted_clauses=True

tm = TMClassifier(
    number_of_clauses=num_clauses,
    T=T,
    s=s,
    max_included_literals=max_included_literals,
    platform=device,
    weighted_clauses=True,
    patch_dim=(patch_size, patch_size),
)

epochs = 100

for epoch in range(epochs):
        start_training = time()
        tm.fit(X_train_thermometer, Y_train)
        stop_training = time()

        start_testing = time()
        Y_test_predicted, Y_test_scores = tm.predict(X_test_thermometer, return_class_sums=True)
        stop_testing = time()

        result_test = 100*(Y_test_scores.argmax(axis=1) == Y_test).mean()

        print("#%d Accuracy: %.2f%% Training: %.2fs Testing: %.2fs" % (epoch+1, result_test, stop_training-start_training, stop_testing-start_testing))

        np.savetxt("2x2ColorThermometers_%d_%d_%d_%.1f_%d_%d_%d.txt" % (epoch+1, num_clauses, T, s, patch_size, max_included_literals, weighted_clauses), Y_test_scores, delimiter=',')


2023-12-10 21:22:11,570 - tmu.clause_bank.clause_bank_cuda - DEBUG - [0.150514ms] - Compiling CUDA Module 'cuda/calculate_clause_outputs_predict.cu'.
2023-12-10 21:22:11,717 - tmu.clause_bank.clause_bank_cuda - DEBUG - [0.145312ms] - Compiling CUDA Module 'cuda/calculate_clause_outputs_update.cu'.
2023-12-10 21:22:11,856 - tmu.clause_bank.clause_bank_cuda - DEBUG - [0.138530ms] - Compiling CUDA Module 'cuda/clause_feedback.cu'.
2023-12-10 21:22:11,997 - tmu.clause_bank.clause_bank_cuda - DEBUG - [0.139715ms] - Compiling CUDA Module 'cuda/tools.cu'.
2023-12-10 21:22:12,147 - tmu.clause_bank.clause_bank_cuda - INFO - Loading compiled CUDA module from '/tmp/tm_kernels/52134d235408ae7d986c11e6c5322c7ae2ae7d57.ptx'.
2023-12-10 21:22:12,150 - tmu.clause_bank.clause_bank_cuda - INFO - Loading compiled CUDA module from '/tmp/tm_kernels/d8627150433a0d735a153a115f7dbe82e2f4194c.ptx'.
2023-12-10 21:22:12,151 - tmu.clause_bank.clause_bank_cuda - INFO - Loading compiled CUDA module from '/tmp/tm_ke